In [2]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.spatial.transform import Rotation as R
from numpy import linalg as LA

In [6]:
class sgeom_coordinate:
    def __init__(
        self,
        clen,
        poni,
        pixel_size,
        X1=[0, 0, 1],
        X2=[1, 0, 0],
        X3=[0, 1, 0],
        rot=[0.0, 0.0, 0.0],
    ):
        self.clen = clen
        self.poni = poni
        self.pixel_size = pixel_size

        # Lab direction vectors by sample vectors
        self.X1_hat = np.array(X1) / LA.norm(X1)  # Sample z
        self.X2_hat = np.array(X2) / LA.norm(X2)  # Sample x
        self.X3_hat = np.array(X3) / LA.norm(X3)  # Sample y
        # Det coordinate expressed by lab vectors
        d1_hat_lab = np.array([1, 0, 0])
        d2_hat_lab = np.array([0, 1, 0])
        d3_hat_lab = np.array([0, 0, 1])
        r = R.from_rotvec([[-rot[0], 0, 0], [0, -rot[1], 0], [0, 0, rot[2]]])
        for i, rotation in enumerate(r):
            d1_hat_lab = rotation.apply(d1_hat_lab)
            d2_hat_lab = rotation.apply(d2_hat_lab)
            d3_hat_lab = rotation.apply(d3_hat_lab)
        self.d1_hat_lab = d1_hat_lab
        self.d2_hat_lab = d2_hat_lab
        self.d3_hat_lab = d3_hat_lab

    def detector_to_lab_coords(self, det_pixels: list[list[int]]):
        converted = []
        for pixel in det_pixels:
            pixel_info = pixel + [0]
            without_rotate = np.array([self.pixel_size, self.pixel_size, 0]) * (
                pixel_info
            ) + np.array([-self.poni[0], -self.poni[1], self.clen])

            converted.append(
                without_rotate[0] * self.d1_hat_lab
                + without_rotate[1] * self.d2_hat_lab
                + without_rotate[2] * self.d3_hat_lab
            )
        return converted

    def lab_to_sample_coords(self, lab_coords: list[list[int | float]]):
        return [
            coord[0] * self.X1_hat + coord[1] * self.X2_hat + coord[2] * self.X3_hat
            for coord in lab_coords
        ]

    def detector_to_sample_coords(self, det_pixels: list[list[int]]):
        return self.lab_to_sample_coords(self.detector_to_lab_coords(det_pixels))

In [ ]:
coord = sgeom_coordinate(
    X1=[0, 0, 1],
    X2=[1, 0, 0],
    X3=[0, 1, 0],
    rot=np.pi * np.array([0.5, 0, 0]),
    clen=0.13,
    poni=[0.12, 0.12],
    pixel_size=2e-4,
)
print((lab := coord.detector_to_lab_coords(det_pixels=[[0, 0], [1, 1]])))
print(coord.lab_to_sample_coords(lab))

In [3]:
photon_energy = 9.3 # keV
wavelength = 12.398/photon_energy # Angstrom
clen = 0.13 # meter
# rotation in radian
rot1 = 0.0
rot2 = 0.0
rot3 = 0.0
rot3 = np.pi/2
rot2 = np.pi/4

In [9]:
sgeom = sgeom_coordinate(rot=[rot1,rot2,rot3])
print(sgeom.d1_hat_lab, sgeom.d2_hat_lab, sgeom.d3_hat_lab)

[1.57009246e-16 7.07106781e-01 7.07106781e-01] [-1.00000000e+00  2.22044605e-16  0.00000000e+00] [-1.57009246e-16 -7.07106781e-01  7.07106781e-01]


In [13]:
# PONI in detector system
det_size = np.array([1200, 1200]) # in pixel
pixel_size = 2e-4 # meter
poni_det = np.append(det_size//2,0)
print(poni_det)
beam_center = []

[600 600   0]
